### 필요한 모듈 불러오기

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
warnings.filterwarnings(action='ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'

import time
from tqdm.notebook import tqdm

import datetime
from datetime import timedelta

### 데이터 불러오기

In [37]:
data =pd.read_json('test_data_v1_COM.json',orient='table')

In [39]:
del data['취급액']

In [40]:
for i in range(0,len(data)):
    if data["브랜드"][i]=="Nobrand":
        data["브랜드"][i]=np.nan

In [41]:
data.dropna(inplace=True) ## 브랜드가 없는 행은 지우고 병합 --> 추후 다시 병합예정

### 브랜드 상품명 검색률 추가

In [43]:
import copy

In [44]:
a = copy.deepcopy(data[data.columns.difference(["상품종류","브랜드상품"])])
len(a)

2712

In [45]:
data.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '브랜드', '상품종류',
       '브랜드상품', '방송날짜', '방송_월', '방송_일', '판매일수'],
      dtype='object')

In [46]:
a["방송날짜"] = pd.to_datetime(a['방송날짜']).dt.tz_localize(None)

In [47]:
import glob
final = pd.DataFrame(columns=a.columns) ## 최종데이터
상품군 = pd.unique(a["상품군"])
a["방송날짜"] = pd.to_datetime(a["방송날짜"])

for i in tqdm(range(0,len(상품군))):
    ## 파일 실행 위치에 맞게 변경
    path = r'C:/Users/SEC/Desktop/매진임박/브랜드상품검색_TEST/'+상품군[i]
    all_files = glob.glob(path + "/*.xlsx")
    noth = pd.DataFrame(columns=["방송날짜","검색률","브랜드"])
    for filename in all_files:
        df = pd.read_excel(filename, header=6)
        new = pd.DataFrame({"방송날짜" : df["날짜"][-1:], "검색률" : df[df.columns[1]][-8:-1].mean(),"브랜드":df.columns[1]})
        noth = pd.concat([noth,new])    

    
    mg1 = a[a["상품군"]==상품군[i]][a["브랜드"]==noth["브랜드"].unique().tolist()[0]]
    mg1["방송날짜"] = pd.to_datetime(mg1["방송날짜"])
    mg2 = noth[noth["브랜드"]==noth["브랜드"].unique().tolist()[0]].reset_index(drop=True)
    mg2["방송날짜"] = pd.to_datetime(mg2["방송날짜"])
    new = pd.merge(mg1,mg2,how='left').fillna(method='ffill')
    merge = pd.DataFrame(columns=new.columns)
    merge = pd.concat([merge,new])
    

    for j in noth["브랜드"].unique().tolist()[1:]:
        mg1 = a[a["상품군"]==상품군[i]][a["브랜드"]==j]
        mg1["방송날짜"] = pd.to_datetime(mg1["방송날짜"])
        mg2 = noth[noth["브랜드"]==j].reset_index(drop=True)
        mg2["방송날짜"] = pd.to_datetime(mg2["방송날짜"])
        new = pd.merge(mg1,mg2,how='left').fillna(method='ffill')
        merge = pd.concat([merge,new])
        
    final = pd.concat([final,merge])
    print(len(merge), 상품군[i])

161 의류
329 속옷
167 이미용
272 주방
260 농수축
191 무형
489 가전
339 생활용품
159 가구
277 잡화
18 건강기능
72 침구



In [48]:
len(final)

2734

In [53]:
total = pd.merge(a,final,how='left').drop_duplicates()

### 원본 데이터와 total 데이터 병합 후 브랜드이름 없는 행의 결측치 채우기

In [55]:
data2 =pd.read_json('test_data_v1_COM.json',orient='table')

data2["방송날짜"] = pd.to_datetime(data2["방송날짜"]).dt.tz_localize(None)

data_a = pd.merge(data2,total,how='left')

- 브랜드 결측치   --> Nobrand
- 판매일수 결측치 --> 해당 상품군의 판매일수로 대체
- 검색률 결측치   --> fill_mean_func 함수를 통해 해당하는 상품군의 평균치로 대체

In [58]:
fill_mean_func = lambda g: g.fillna(g.mean())

data_a = data_a.groupby('상품군').apply(fill_mean_func)
data_a.reset_index(drop=True,inplace=True)

In [59]:
len(data_a)
data_a.isnull().sum()

2891

방송일시        0
노출(분)       0
마더코드        0
상품코드        0
상품명         0
상품군         0
판매단가        0
취급액      2891
브랜드         0
상품종류        0
브랜드상품       0
방송날짜        0
방송_월        0
방송_일        0
판매일수        0
검색률         0
dtype: int64

### 최종 데이터

In [62]:
data_a.rename(columns={'검색률':'검색률2'},inplace=True)

In [63]:
data_a.to_json("test_data_v2_COM.json",orient ="table")